# Prediction with LSTM model (v4.6.1)
<p style='text-align: right;'>with selectd6.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용
  * 2025/05/27
  * 2025/05/30 : v4.5.1 작성 시작
  * 2025/06/04 : v4.5.2 last update
  * 2025/06/04 : v4.6.1 작성 시작

In [ ]:
!pip install scikit-learn
!pip install tensorflow
!pip install numpy pandas keras scikit-learn matplotlib scikeras
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [1]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

import os
import sys
from datetime import datetime
cwd = os.getcwd()
sys.path.append(cwd)
print(sys.path)

restart kernel... 2025-06-04 10:51:35.034677
['/home/swhors/jupyter-workspace/LSTM/$PWD', '/home/swhors/jupyter-workspace/finance', '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-paches', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages', '/home/swhors/jupyter-workspace/LSTM']


In [8]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model, generate_metric
from lib.common_env_sets import lstm_units, dense_units, steps, metrics, dropout
from lib.common_env_sets import learning_rate, last_lstm_return_sequences
from lib.common_env_sets import loss, output_dense_activation, epochs, rand_seed


print(f"imported library. ({datetime.now()})")

imported library. (2025-06-06 02:03:46.764980)


In [9]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd6.csv'

mode='predict' # help = "back-test or predict")

mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
training_length=0.95 # default = 0.9)
from_pos = 0
last = [[3, 4, 6, 8, 32, 42],
        [8, 11, 14, 17, 36, 39],
        [1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")


finished to set environemnt. (2025-06-06 02:03:53.599804)


In [10]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }

print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-06-06 02:03:55.559866


In [11]:
###########
## layers
## LSTM Neural 계층 선언
###########
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

sel_date = datetime(2025, 5, 31, 20, 35)
rans_seed = (sel_date.timestamp() / 10000000)

lstm_args = {14: {"model":{"learning_rate": learning_rate[0],
                           "dropout": dropout[0],
                           "steps": steps[0], # last output elements count
                           "metrics": metrics[1],
                           "last_lstm_return_sequences": last_lstm_return_sequences[0],
                           "lstm_units": lstm_units[2],
                           "dense_units": dense_units[2],
                           "output_dense_activation": output_dense_activation[3],
                           "loss": loss[0],
                           "return_state": False,
                           "rand_seed": rans_seed # rand_seed[0]
                           },
                 "train": {"epochs": epochs[2],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 0 #100
                           },
                 "dataset": datasets[0]
                 },
             44: {"model":{"learning_rate": learning_rate[0],
                           "dropout": dropout[0],
                           "steps": steps[0], # last output elements count
                           "metrics": metrics[0],
                           "last_lstm_return_sequences": last_lstm_return_sequences[1],
                           "lstm_units": lstm_units[2],
                           "dense_units": dense_units[2],
                           "output_dense_activation": output_dense_activation[3],
                           "loss": loss[0],
                           "return_state": False,
                           "rand_seed": rans_seed # rand_seed[0]
                           },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 0 #100
                           },
                 "dataset": datasets[0]
                 },
             }
lstm_args_4 = {4: {"model":{"learning_rate": learning_rate[0],
                            "dropout": dropout[1],
                            "steps": steps[0], # last output elements count
                            "metrics": metrics[0],
                            "last_lstm_return_sequences": last_lstm_return_sequences[0],
                            "lstm_units": lstm_units[2],
                            "dense_units": dense_units[2],
                            "output_dense_activation": output_dense_activation[3],
                            "loss": loss[1],
                            "return_state": False,
                            "rand_seed": rans_seed # rand_seed[0]
                            },
                   "train": {"epochs": epochs[3],
                             "is_shuffle": False,
                             "batch": 3,
                             "steps_per_epoch": 90 #100
                             },
                   "dataset": datasets[0]
                   },
             }
lstm_args_14 = {14: {"model":{"learning_rate": learning_rate[0],
                              "dropout": dropout[0],
                              "steps": steps[0], # last output elements count
                              "metrics": metrics[1],
                              "last_lstm_return_sequences": last_lstm_return_sequences[0],
                              "lstm_units": lstm_units[2],
                              "dense_units": dense_units[2],
                              "output_dense_activation": output_dense_activation[3],
                              "loss": loss[0],
                              "return_state": False,
                              "rand_seed": rans_seed # rand_seed[0]
                              },
                     "train": {"epochs": epochs[2],
                               "is_shuffle": False,
                               "batch": 1,
                               "steps_per_epoch": 0 #100
                               },
                     "dataset": datasets[0]
                     },
                }

lstm_args_44 = {44: {"model":{"learning_rate": learning_rate[0],
                              "dropout": dropout[0],
                              "steps": steps[0], # last output elements count
                              "metrics": metrics[0],
                              "last_lstm_return_sequences": last_lstm_return_sequences[1],
                              "lstm_units": lstm_units[2],
                              "dense_units": dense_units[2],
                              "output_dense_activation": output_dense_activation[3],
                              "loss": loss[0],
                              "return_state": False,
                              "rand_seed": rans_seed # rand_seed[0]
                              },
                     "train": {"epochs": epochs[3],
                               "is_shuffle": False,
                               "batch": 1,
                               "steps_per_epoch": 0 #100
                               },
                     "dataset": datasets[0]
                     },
                }

lstm_args_2 = {2: {"model":{"learning_rate": learning_rate[0],
                            "dropout": dropout[0],
                            "steps": steps[0], # last output elements count
                            "metrics": metrics[0],
                            "last_lstm_return_sequences": last_lstm_return_sequences[0],
                            "lstm_units": lstm_units[1],
                            "dense_units": dense_units[1],
                            "output_dense_activation": output_dense_activation[0],
                            "loss": loss[0],
                            "rand_seed": rans_seed
                            },
                   "train": {"epochs": epochs[1],
                             "is_shuffle": False,
                             "batch": 1,
                             "steps_per_epoch": 1
                             },
                   "dataset": datasets[0]
                   },
             }

lstm_args_6 = {6: {"model":{"learning_rate": learning_rate[0],
                            "dropout": dropout[1],
                            "steps": steps[0], # last output elements count
                            "metrics": metrics[0],
                            "last_lstm_return_sequences": last_lstm_return_sequences[0],
                            "lstm_units": lstm_units[5],
                            "dense_units": dense_units[4],
                            "output_dense_activation": output_dense_activation[3],
                            "loss": loss[1],
                            "return_state": False,
                            "lstm_model": "Bidirectional",
                            "rand_seed": rans_seed # rand_seed[0]
                            },
                   "train": {"epochs": epochs[3],
                             "is_shuffle": False,
                             "batch": 3,
                             "steps_per_epoch": 90 #100
                             },
                   "dataset": datasets[0]
                   }
             }

print(f'completed to set env for all models. {datetime.now()}')

completed to set env for all models. 2025-06-06 02:03:57.686414


In [12]:
def gen_metric(test_id, gen_num, round_limit, lstm_args=lstm_args):
    """ gen_metric """
    model_version_from = 1
    model_version_end = 1
    print(f'gen_metric.step.0 [{datetime.now()}]')
    args = {'test_id': test_id,
            'version': model_version_from,
            'write_to_db': True,
            'last': last,
            'mode2': mode2,
            'gen_num': gen_num,
            'round_limit': round_limit,
            'db_file_name': "./db/metrics.db"
            }
    print(f'gen_metric.step.1 [{datetime.now()}]')
    generate_metric(args=args,
                    from_version=model_version_from,
                    to_version=model_version_end,
                    lstm_args=lstm_args)
    print(f'gen_metric.step.2 [{datetime.now()}]')

In [6]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id = 'P4_6_1_1'
gen_num = 1
round_limit = 1000

gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

Test Start: 1~1
CPU 시간: 4.295898 초
메모리 사용량: 616144 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 15:41:39.300424
********************
start to train all models. 2025-06-05 15:41:39.300454
14's training. status=start 2025-06-05 15:41:39.300470
14's training. status=end 2025-06-05 15:42:29.403099
44's training. status=start 2025-06-05 15:42:29.403201
44's training. status=end 2025-06-05 15:44:00.291719
completed to train all models. 2025-06-05 15:44:00.291818
train to create model(1). 2025-06-05 15:44:03.291901
CPU 시간: 444.326674 초
메모리 사용량: 1066912 KB


In [20]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id = 'P4_6_1_2'
gen_num = 1
round_limit = 1000

gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit, lstm_args=lstm_args_14)

gen_metric.step.0 [2025-06-05 16:35:17.115076]
gen_metric.step.1 [2025-06-05 16:35:17.115151]
Test Start: 1~1
CPU 시간: 887.653416 초
메모리 사용량: 1162592 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 16:35:17.115241
********************
start to train all models. 2025-06-05 16:35:17.115260
14's training. status=start 2025-06-05 16:35:17.115277
14's training. status=end 2025-06-05 16:36:06.258793
completed to train all models. 2025-06-05 16:36:06.259092
train to create model(1). 2025-06-05 16:36:09.259175
CPU 시간: 1085.954814 초
메모리 사용량: 1259808 KB
gen_metric.step.2 [2025-06-05 16:39:02.396592]


In [12]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id = 'P4_6_1_3'
gen_num = 1
round_limit = 1000

gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit, lstm_args=lstm_args_44)

gen_metric.step.0 [2025-06-05 15:54:24.916540]
gen_metric.step.1 [2025-06-05 15:54:24.916597]
Test Start: 1~1
CPU 시간: 643.848371 초
메모리 사용량: 1069264 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 15:54:24.916656
********************
start to train all models. 2025-06-05 15:54:24.916666
44's training. status=start 2025-06-05 15:54:24.916676
44's training. status=end 2025-06-05 15:55:54.581077
completed to train all models. 2025-06-05 15:55:54.581205
train to create model(1). 2025-06-05 15:55:57.581298
CPU 시간: 886.626137 초
메모리 사용량: 1162592 KB
gen_metric.step.2 [2025-06-05 15:58:51.713842]


In [23]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id = 'P4_6_1_4'
gen_num = 1
round_limit = 1000

gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit, lstm_args=lstm_args_4)

gen_metric.step.0 [2025-06-05 18:03:06.790972]
gen_metric.step.1 [2025-06-05 18:03:06.791049]
Test Start: 1~1
CPU 시간: 1087.019035 초
메모리 사용량: 1259808 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 18:03:06.791137
********************
start to train all models. 2025-06-05 18:03:06.791153
4's training. status=start 2025-06-05 18:03:06.791167
4's training. status=end 2025-06-05 18:04:01.033005
completed to train all models. 2025-06-05 18:04:01.033137
train to create model(1). 2025-06-05 18:04:04.033231
CPU 시간: 1281.096318 초
메모리 사용량: 1395776 KB
gen_metric.step.2 [2025-06-05 18:06:54.907379]


In [23]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id = 'P4_6_1_5'
gen_num = 1
round_limit = 1000

gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit, lstm_args=lstm_args_2)

gen_metric.step.0 [2025-06-05 18:03:06.790972]
gen_metric.step.1 [2025-06-05 18:03:06.791049]
Test Start: 1~1
CPU 시간: 1087.019035 초
메모리 사용량: 1259808 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 18:03:06.791137
********************
start to train all models. 2025-06-05 18:03:06.791153
4's training. status=start 2025-06-05 18:03:06.791167
4's training. status=end 2025-06-05 18:04:01.033005
completed to train all models. 2025-06-05 18:04:01.033137
train to create model(1). 2025-06-05 18:04:04.033231
CPU 시간: 1281.096318 초
메모리 사용량: 1395776 KB
gen_metric.step.2 [2025-06-05 18:06:54.907379]


In [13]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id = 'P4_6_1_6'
gen_num = 1
round_limit = 1000

gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit, lstm_args=lstm_args_6)

gen_metric.step.0 [2025-06-06 02:04:06.727261]
gen_metric.step.1 [2025-06-06 02:04:06.727746]
Test Start: 1~1
CPU 시간: 1.78125 초
메모리 사용량: 374760 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:04:06.727820
********************
start to train all models. 2025-06-06 02:04:06.727829
6's training. status=start 2025-06-06 02:04:06.727837
6's training. status=end 2025-06-06 02:04:16.525473
completed to train all models. 2025-06-06 02:04:16.525559
train to create model(1). 2025-06-06 02:04:19.525649
INFO:tensorflow:Assets written to: ram://4feacfa172b64cebbc81751509f72303/assets


INFO:tensorflow:Assets written to: ram://4feacfa172b64cebbc81751509f72303/assets


CPU 시간: 35.375 초
메모리 사용량: 1056312 KB
gen_metric.step.2 [2025-06-06 02:05:25.607029]


In [ ]:
## end